# Practice PS07: Outlier analysis
<font size="+2" color="blue">Additional results: faster method to find all depths</font>

Author: <font color="blue">Aniol Petit</font>

E-mail: <font color="blue">aniol.petit01@estudiant.upf.edu</font>

Date: <font color="blue">14/11/2024</font>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import io
import random
import numpy as np

# 1. Dataset

In [ ]:
# LEAVE AS-IS

df = pd.read_csv('annthyroid.csv', sep=',')

# Replace the "label" column by an "abnormal" column
df['result'] = df['label'].apply(lambda x: 'abnormal' if x == 1 else 'normal')
df.drop(columns=['label'], inplace=True)

<font size="+1" color="red">Replace this cell with your code to print the number and percentages of patients in each group.</font>

In [ ]:
total = len(df)
print(f"Total patients: {total}")
normal = len(df[df["result"]=="normal"])
print(f"Normal thyroid: {normal} ({round((normal/total)*100, 2)}%)")
abnormal = len(df[df["result"]=="abnormal"])
print(f"Normal thyroid: {abnormal} ({round((abnormal/total)*100, 2)}%)")

<font size="+1" color="red">Replace this cell with your code to print the 10 normal exams and the centroid of all normal exams, and the first 10 abnormal exams and the centroid of all abnormal exams.</font>

In [ ]:
normal_exams = df[df["result"] == "normal"]
print("First 10 normal exams:")
display(normal_exams.head(10))

abnormal_exams = df[df["result"] == "abnormal"]
print("First 10 abnormal exams:")
display(abnormal_exams.head(10))

normal_centroid = normal_exams.drop(columns=["result"]).mean()
print(f"Centroid of a normal exam: \n{normal_centroid}\n")

abnormal_centroid = abnormal_exams.drop(columns=["result"]).mean()
print(f"Centroid of an abnormal exam: \n{abnormal_centroid}")

<font size="+1" color="red">Replace this cell with a brief comment indicating, based only on the centroids, which features do you think could be useful for differentiating between normal and abnormal thyroids? Why?</font>

In [ ]:
mean_differences = abs(normal_centroid - abnormal_centroid)
print(mean_differences)

Based on the centroid values for normal and abnormal exams, we can see some differences in certain features mean value that lead us to thinking that they can be relevant for differentiating normal from abnormal exams. In this case, these features would be **f1**, **f2**, **f4** and **f6**. These differences, although they are not very big, they are quite more significant than in the rest of features, which tells us that they can help us tell the result of an exam. The most relevant among these would be **f6**.

<font size="+1" color="red">Replace this cell with your code to create a scatter matrix as described above.</font>

In [ ]:
color_code = {'abnormal': 'red', 'normal': 'blue'}
colors = df['result'].apply(lambda x: color_code[x])
plt.figure(figsize=(12, 12))  # Set the figure size
pd.plotting.scatter_matrix(df, c=colors, figsize=(12, 12))
plt.suptitle("Scatter Matrix of Thyroid Exam Features", size=16)
plt.show()

<font size="+1" color="red">Replace this cell with a brief commentary on whether you already guess any difference between normal and abnormal exams, based on combinations of features. What are the differences you can appreciate?</font>

In combinations like (f1, f2), (f1, f4), and (f1, f6), there is a noticeable separation between the red (abnormal) and blue (normal) points. Abnormal exams tend to cluster differently in these dimensions, often forming smaller, distinct groups away from the main cluster of normal exams.

Abnormal exams (red points) appear to fall toward the edges of the feature distributions in several plots, especially in features like f2 and f3. This suggests that these features might have threshold values or unusual ranges that indicate abnormality.

Normal exams (blue points) tend to form larger, denser clusters in most plots, suggesting they occupy a more consistent range across multiple features, whereas abnormal exams have more variation and outliers across different dimensions.

# 1. Create isolation tree

## 1.1. Random dimension and random split

<font size="+1" color="red">Replace this cell with code implementing "pick_random_dimension" and a couple of calls to this function.</font>

In [ ]:
def pick_random_dimension(df, columns_to_ignore):
    available_columns = [col for col in df.columns if col not in columns_to_ignore]     
    return random.choice(available_columns)

columns_to_ignore = ['id',
                     'result']
test1 = pick_random_dimension(df, columns_to_ignore)
test2 = pick_random_dimension(df, columns_to_ignore)
print(f"Test 1: {test1}")
print(f"Test 2: {test2}")

<font size="+1" color="red">Replace this cell with code implementing "pick_random_split" and a couple of calls to test this function; each call should return a random split point chosen uniformly at random between the minimum and the maximum value along a dimension.</font>

In [ ]:
def pick_random_split(df, dimension):
    min_value = np.min(df[dimension])
    max_value = np.max(df[dimension])
    return random.uniform(min_value, max_value)

test1 = pick_random_split(df, "f2")
test2 = pick_random_split(df, "f6")
print(f"Test 1: {test1}")
print(f"Test 2: {test2}")

## 1.2. Implement split data into two

<font size="+1" color="red">Replace this cell with code implementing "split_dataset".</font>

In [ ]:
def split_dataset(df, dimension, split):
    smaller_df = df[df[dimension] <= split]
    greater_df = df[df[dimension] > split]
    return smaller_df, greater_df

<font size="+1" color="red">Replace this cell with code testing "split_dataset", i.e., select a dimension and a cut-off and split the dataset, then print the two pieces and check that it is working correctly. Also check that the sum of the sizes of the two pieces is the size of the original data.</font>

In [ ]:
dimension = 'f1'
split = pick_random_split(df, dimension)
subset_less_equal, subset_greater = split_dataset(df, dimension, split)

print(f"Selected dimension: {dimension}")
print(f"Split point: {split}")
print("\nSubset with dimension <= split:")
display(subset_less_equal)
print("\nSubset with dimension > split:")
display(subset_greater)

original_size = len(df)
subset_sizes_sum = len(subset_less_equal) + len(subset_greater)

print(f"\nOriginal dataset size: {original_size}")
print(f"Sum of sizes of the two subsets: {len(subset_less_equal)} + {len(subset_greater)} = {subset_sizes_sum}")
print("Test passed!" if original_size == subset_sizes_sum else "Test failed!")

## 1.3. Create one isolation tree

In [ ]:
# LEAVE AS-IS

dfi = df.copy()
dfi.insert(0, 'id', [("P%.4d" % x) for x in range(1, 1 + len(df))])
dfi

<font size="+1" color="red">Replace this cell with you code implementing "isolation_tree".</font>

In [ ]:
def isolation_tree(data, columns_to_ignore, min_items):
    tree = {}
    if len(data) <= min_items:
        return {"contents": data}
    else:
        dimension = pick_random_dimension(data, columns_to_ignore) 
        split = pick_random_split(data, dimension)
        left_data, right_data = split_dataset(data, dimension, split)
        left = isolation_tree(left_data, columns_to_ignore, min_items)
        right = isolation_tree(right_data, columns_to_ignore, min_items)
        tree = {"left": left, "right": right, "dimension": dimension, "split": split}
        return tree  

In [ ]:
# LEAVE AS-IS

mytree = isolation_tree(dfi, ['id', 'result'], 2000)

In [ ]:
# LEAVE AS-IS

class tree_drawing(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __str__(self, label='', level=0):
        ret = "-" * level + label + ":" + repr(self.value)+"\n"
        if self.left:
            ret += self.left.__str__("<", level+1)
        if self.right:
            ret += self.right.__str__(">", level+1)
        return ret

    def __repr__(self):
        return self.value
    
def tree_to_nodes(tree):
    if 'contents' in tree:
        data = tree['contents']
        normal_count = len(data[data['result'] == 'normal'])
        abnormal_count = len(data[data['result'] == 'abnormal'])
        description = "external node: %d normal + %d abnormal" % (normal_count, abnormal_count)
        return tree_drawing(description)
    else:
        left = tree['left']
        right = tree['right']
        description = 'internal node: ' + tree['dimension'] + ' <= ' + ("%.2f" % tree['split'])
        n = tree_drawing(description, tree_to_nodes(left), tree_to_nodes(right) )
        return n

In [ ]:
print(tree_to_nodes(mytree))

<font size="+1" color="red">Replace this cell with you code implementing `get_max_tree_depth`</font>

In [ ]:
def get_max_tree_depth(tree):
    if "contents" in tree:
        return 1
    left_depth = get_max_tree_depth(tree["left"])
    right_depth = get_max_tree_depth(tree["right"])
    return 1 + max(left_depth, right_depth)


<font size="+1" color="red">Replace this cell with you code testing `get_max_tree_depth` on `mytree`.</font>

In [ ]:
max_depth = get_max_tree_depth(mytree)
print(f"Maximum depth of the isolation tree: {max_depth}")

## 2.1. Create an isolation forest

<font size="+1" color="red">Replace this cell with you code implementing "isolation_tree".</font>

In [ ]:
def isolation_forest(dfi, columns_to_ignore, min_items, num_trees):
    trees = []
    for _ in range(num_trees):
        trees.append(isolation_tree(dfi, columns_to_ignore, min_items))
    return trees

In [ ]:
# MODIFY IF YOU SEE THAT IT IMPROVES THE SEPARATION BETWEEN CLASSES

min_items = 50
num_trees = 20

In [ ]:
# LEAVE AS-IS

myforest = isolation_forest(dfi, ['id', 'result'], min_items, num_trees)
print("The forest has %d trees" % len(myforest))

In [ ]:
# LEAVE AS-IS

for i in range(0, 2):
    print("Tree number %d" % i)
    print(tree_to_nodes(myforest[i]))
    print()

## 2.2. Find the average depth of an item

In [ ]:
# LEAVE AS-IS

def find_element_depth_aux(tree, column_name, column_value, depth):
    
    if 'contents' in tree:
        # We are in a leaf node
        
        data = tree['contents']
        matching = data[data[column_name] == column_value]
        if len(matching) == 0:
            return None
        elif len(matching) > 1:
            print("Error: multiple elements match your search criterion")
            return None
        else:
            return depth
    else:
        # We are in an internal node
        
        in_left = find_element_depth_aux(tree['left'], column_name, column_value, depth+1)
        if in_left == None:
            in_right = find_element_depth_aux(tree['right'], column_name, column_value, depth+1)
            return in_right
        else:
            return in_left
        
def find_element_depth(tree, column_name, column_value):
    return find_element_depth_aux(tree, column_name, column_value, 0)

In [ ]:
# LEAVE AS-IS

def print_sample_depths(df, sample_size, tested_tree):

    for result in ['normal', 'abnormal']:
        print("Depths for %s items" % result)

        sample = list(df[df['result'] == result]['id'].sample(sample_size))

        max_depth = get_max_tree_depth(tested_tree)
        for random_id in sample:
            print("- id=%s is at depth %d/%d" % (random_id, find_element_depth(tested_tree, "id", random_id), max_depth))
            
    
print("On shallow tree, having maximum depth %d" % get_max_tree_depth(mytree))
print_sample_depths(dfi, 5, mytree)
print()

print("On first tree of forest, having maximum depth %d" % get_max_tree_depth(myforest[0]))
print_sample_depths(dfi, 5, myforest[0])
print()


<font size="+1" color="red">Replace this cell with a brief commentary on these depths.</font>

**Shallow Tree (Max Depth: 16):**
Normal items tend to have relatively deep depths (ranging from 4 to 13), which suggests that normal data points are more spread out across the tree and may require more splits to isolate them.
Abnormal items, on the other hand, are often found at shallower depths (ranging from 2 to 10). This indicates that abnormal points tend to be isolated faster, as the isolation tree's purpose is to split off anomalies quickly with fewer decisions.

**First Tree in the Isolation Forest (Max Depth: 43):**
Normal items in the first tree also exhibit a wider range of depths (from 16 to 27), with some items being deep in the tree, suggesting more complex splits for normal cases.
Abnormal items show more variation in their depths, but still tend to be isolated at lower depths (3 to 16). This continues to reinforce the idea that anomalies are generally isolated earlier (with fewer splits) compared to normal instances.

<font size="+1" color="red">Replace this cell with you code implementing "find_average_depth".</font>

In [ ]:
def find_average_depth(forest, column_name, value):
    depths = []
    
    for tree in forest:
        depth = find_element_depth(tree, column_name, value)
        if depth is not None:  # If the element exists in the tree
            depths.append(depth)
    
    if len(depths) == 0:
        return None  # Element not found in any tree
    else:
        return sum(depths) / len(depths) 

In [ ]:
# LEAVE AS-IS

for result in ['normal', 'abnormal']:
    print("Average depths for %s items" % result)
    
    sample = list(dfi[dfi['result'] == result]['id'].sample(5))
    
    for random_id in sample:
        print("- id=%s is at average depth %.1f" % (random_id, find_average_depth(myforest, "id", random_id)))
    
    print()

<font size="+1" color="red">Replace this cell with a brief commentary indicating how would you make the "find_average_depth" function more efficient if you wanted to obtain the average depth of **all** elements. Be concise but be precise, providing pseudocode if you consider it clearer than a text explanation.</font>

To make the find_average_depth function more efficient for obtaining the average depth of all elements (rather than just one element), we can modify the approach to process all unique elements in the dataset, calculating their average depth across the trees in the forest.
Instead of calculating the depth for each element in each tree separately, we can first traverse the dataset to collect the unique values for the column_name, and then calculate the average depth for all elements in the forest at once.

# 3. Find the average depth of a sample of normal and abnormal points

## 3.1. Determine average depths

In [ ]:
# LEAVE AS-IS

abnormal_point_ids = list(dfi[dfi['result'] == 'abnormal']['id'].sample(30))
normal_point_ids = list(dfi[dfi['result'] == 'normal']['id'].sample(30))

print("Normal point sample ids   : %s" % normal_point_ids)
print("Abnormal point sample ids : %s" % abnormal_point_ids)

In [ ]:
# LEAVE AS-IS

normal_depths = []
print("Depths of items in the 'normal' sample:")
for i in normal_point_ids:
    depth = find_average_depth(myforest, 'id', i)
    normal_depths.append(depth)
    print(" element id=%s is at average depth %.2f" % (i, depth))
print()

In [ ]:
# LEAVE AS-IS

abnormal_depths = []
print("Depths of items in the 'abnormal' sample:")
for i in abnormal_point_ids:
    depth = find_average_depth(myforest, 'id', i)
    abnormal_depths.append(depth)
    print(" element id=%s is at average depth %.2f" % (i, depth))
print()

In [ ]:
# LEAVE AS-IS

plt.figure(figsize=(12,4))
plt.hist(normal_depths, label='Normal', bins=15, density=False, histtype='step', color='blue')
plt.hist(abnormal_depths, label='Abnormal', bins=15, density=False, histtype='step', color='red')
plt.xlabel('Average depth in the isolation forest')
plt.ylabel('Frequency')
plt.legend()
plt.show()

<font size="+1" color="red">Replace this cell with a brief commentary of what you see in this histogram.</font>

Both normal and abnormal items span a similar range of depths, especially in the mid-range (approximately 10-20), with significant overlap. This overlap suggests that depth alone does not perfectly distinguish between normal and abnormal exams, indicating that some abnormal exams have similar average depths to normal ones, and vice versa.

The abnormal items show a higher frequency at shallower depths (around 5-15). Shallower depths imply higher isolation (i.e., more outlier-like behavior), which aligns with the expectation that abnormal items might be isolated earlier in the forest.

The normal items are more frequently found at deeper depths, particularly beyond 20. This is consistent with the intuition that normal items blend more with the rest of the dataset, requiring more splits to isolate them.

## 3.2. Compute outlier scores and conclude

In [ ]:
# LEAVE AS-IS

# Source: https://stackoverflow.com/a/27683292/1235784

from numpy import euler_gamma
from scipy.special import digamma
def harmonic(s):
    return digamma(s + 1) + euler_gamma

In [ ]:
# LEAVE AS-IS

def outlier_score_factory():
    n = len(df.index)
    c_n = 2.0 * harmonic(n - 1.0) - (2.0 * (n-1)/n)
    return lambda d: 2.0**(-d/c_n)

outlier_score = outlier_score_factory()

normal_scores = [outlier_score(d) for d in normal_depths]
abnormal_scores = [outlier_score(d) for d in abnormal_depths]

In [ ]:
# LEAVE AS-IS

plt.figure(figsize=(16,3))

# Plot the two histograms; the parameter 'alpha' is the transparency of the bar
plt.hist(normal_scores, label='Normal', bins=20, density=False, histtype='step', color='blue', alpha=0.7)
plt.hist(abnormal_scores, label='Abnormal', bins=20, density=False, histtype='step', color='red', alpha=0.7)

# Draw the axis labels, legend, and display
plt.xlabel('Outlier score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

<font size="+1" color="red">Replace this cell with a brief commentary of what you see in this histogram.</font>

There is some overlap in the outlier scores between normal and abnormal items, particularly in the range from about 0.4 to 0.6. This suggests that some abnormal items are not easily distinguishable from normal items based on outlier score alone.

The normal items tend to have a higher frequency of scores around 0.4 to 0.5, whereas abnormal items show a broader spread with several peaks.
Abnormal items also extend to higher outlier scores (up to around 0.8), which indicates that they are, on average, more isolated in the trees and thus have higher outlier scores.


<font size="+1" color="red">Replace this cell with your code to determine an optimal threshold.</font>

In [ ]:
all_scores = normal_scores + abnormal_scores
min_score, max_score = min(all_scores), max(all_scores)

thresholds = np.linspace(min_score, max_score, 100)

best_threshold = None
best_accuracy = 0

for threshold in thresholds:
    normal_pred = [1 if score <= threshold else 0 for score in normal_scores]  
    abnormal_pred = [1 if score > threshold else 0 for score in abnormal_scores]  
    
    true_negatives = sum(normal_pred) 
    false_positives = len(normal_scores) - true_negatives  
    true_positives = sum(abnormal_pred)  
    false_negatives = len(abnormal_scores) - true_positives  
    
    total = len(normal_scores) + len(abnormal_scores)
    accuracy = (true_positives + true_negatives) / total
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold
        
print(f"Optimal Threshold: {best_threshold}")
print(f"Accuracy at Optimal Threshold: {best_accuracy}")

<font size="+1" color="red">Replace this cell with your evaluation of the optimal threshold.</font>

In [ ]:
fpr = false_positives / len(normal_scores)
fnr = false_negatives / len(abnormal_scores)
print(f"False Positive Rate (FPR): {fpr:.4f}")
print(f"False Negative Rate (FNR): {fnr:.4f}")

## EXTRA POINTS

In [ ]:
from collections import defaultdict

def find_average_depths(forest, column_name):
    # Dictionary to accumulate depths for each element
    depths_dict = defaultdict(list)
    
    # Traverse each tree in the forest
    for tree in forest:
        def record_depths(node, depth):
            if 'contents' in node:
                # Leaf node: Record depth for each element in this node
                for _, row in node['contents'].iterrows():
                    element_id = row[column_name]
                    depths_dict[element_id].append(depth)
            else:
                # Internal node: Recur on both children with increased depth
                record_depths(node['left'], depth + 1)
                record_depths(node['right'], depth + 1)
        
        # Start depth recording from the root node
        record_depths(tree, 0)
    
    # Calculate average depths for each element
    average_depths = {element_id: sum(depths) / len(depths) for element_id, depths in depths_dict.items()}
    return average_depths

# Compute average depths for all elements in the dataset
average_depths = find_average_depths(myforest, column_name="id")


# Separate normal and abnormal scores based on results
normal_depths = [average_depths[id] for id in dfi[dfi['result'] == 'normal']['id']]
abnormal_depths = [average_depths[id] for id in dfi[dfi['result'] == 'abnormal']['id']]

# Calculate outlier scores using the optimized average depths
normal_scores = [outlier_score(d) for d in normal_depths]
abnormal_scores = [outlier_score(d) for d in abnormal_depths]

# Plotting the histogram
plt.figure(figsize=(16,3))

# Plot the two histograms for all elements
plt.hist(normal_scores, label='Normal', bins=20, density=False, histtype='step', color='blue', alpha=0.7)
plt.hist(abnormal_scores, label='Abnormal', bins=20, density=False, histtype='step', color='red', alpha=0.7)

# Draw the axis labels, legend, and display
plt.xlabel('Outlier score')
plt.ylabel('Frequency')
plt.legend()
plt.show()


<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>